In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
%matplotlib inline
homedir = !chdir
folder = r'D:\measuring\data'

In [ ]:
def find_all_relevant_files(folder):
    """
    Finds all purification related data.
    Neglects files that are smaller than a 5 Mb.
    """
    t0 = '20161224_00000'
    filepaths = []
    filenames = []
    filenameouts = []
    fs = []
    measurement_name = 'Purify_XX'
    while tb.latest_data(contains = measurement_name,folder=folder,older_than = t0,raise_exc=False):
        t0,f = tb.latest_data(contains = measurement_name,folder = folder,return_timestamp=True,older_than=t0)
        filepath = tb.get_msmt_fp(f)
        filesize = os.path.getsize(filepath)
        if filesize > 5e6: # file size is given in bytes
            fs.append(f)
            filename = os.path.split(filepath)[1]
            filenameout = filename[:-5]+'_slim.hdf5'
            filepaths.append(filepath);filenames.append(filename);filenameouts.append(filenameout)
    return fs,filepaths,filenames,filenameouts
fs,filepaths,filenames,filenameouts = find_all_relevant_files(folder)
print fs

In [ ]:
### WATCH OUT THIS WILL DELETE ALL PQ DATA FROM THE SELECTED FILES --> NOT RECOVERABLE!
if True:
    for f,filepath,filename,filenameout in zip(fs,filepaths,filenames,filenameouts):
        index = 1
        keys = []#['PQ_hist']
        h5file = h5py.File(filepath,'r+')
        breakstatement = True
        while breakstatement:
            try:
                dataset = h5file['PQ_time-'+str(index)]
            except:
                breakstatement = False
                break
            keys += ['PQ_special-'+str(index),
                'PQ_sync_number-'+str(index),'PQ_sync_time-'+str(index),'PQ_time-'+str(index),'PQ_channel-'+str(index)]
            index +=1

        ### found all the relevant PQ names now we can delete
        for k in keys:
            try:
                del h5file[k]
            except:
                print 'could not delete %s in %s... no photons??' % (k,filename)

        h5file.flush()
        h5file.close()

In [ ]:
for f,filename,filenameout in zip(fs,filenames,filenameouts):
    ### repack files into something more compact
    %cd {f}
    !h5repack {filename} {filenameout}

In [ ]:
for f,filename,filenameout in zip(fs,filenames,filenameouts):
    ### delete unpacled files
    %cd {f}
    !del {filename}

In [ ]:
for f,filename,filenameout in zip(fs,filenames,filenameouts):
    ### rename packed files
    %cd {f}
    !ren {filenameout} {filename}